## Imports

In [163]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from pathlib import Path
#https://pytorch.org/vision/stable/models/generated/torchvision.models.video.r3d_18.html#torchvision.models.video.r3d_18
from torchvision.models.video import r3d_18 as resnet3D

## Load data

### Fmri data

In [2]:
NUM_SUBJS = 8
subjects_fmri = [] #stores all 8 subject fmri np arrays

fMRI_folder = Path('./doi_10.5061_dryad.gt413__v1')
assert fMRI_folder.exists(), f"Foldder: {fMRI_folder} does not exist."

for subj_id in range(8):
#     fmri_file_name = str(subj_id) + '_masked_2d.npy'
#     fmri = np.load(fMRI_folder / fmri_file_name)  
    fmri_file_name = str(subj_id) + '_smooth_nifti_4d.nii'
    fmri = nib.load(fMRI_folder / fmri_file_name)
    fmri = np.array(fmri.dataobj)
    assert isinstance(fmri, np.ndarray), f"Imported fmri_scan for subject {subj_id} is not of type numpy.ndarray"
    assert(fmri.ndim) == 4, f"Imported fmri_scan for subject {subj_id} is not 4 dimensional"
    subjects_fmri.append(fmri)

### Word features

In [3]:
feature_matrix = np.zeros((5176,195)) #stores the feature vectors as a row for each word
feature_names = [] #stores the names of all features in order
feature_types = {} #stores the types of features and all the names of the features for each type

features = sio.loadmat(fMRI_folder / 'story_features.mat')
feature_count = 0
for feature_type in features['features'][0]:
    feature_types[feature_type[0][0]] = []
    if isinstance(feature_type[1][0], str):
        feature_types[feature_type[0][0]].append(feature_type[1][0])
        feature_names.append(feature_type[1][0])
    else:
        for feature in feature_type[1][0]:
            feature_types[feature_type[0][0]].append(feature[0])
            feature_names.append(feature[0])
    feature_matrix[:, feature_count:feature_count+feature_type[2].shape[1]] = feature_type[2] #adds the (5176xN) feature values to the feature matrix for the current feature group
    feature_count += feature_type[2].shape[1]

### Word values and timings

In [4]:
words_info = [] #stores tuples of (word, time, features) sorted by time appeared

mat_file = fMRI_folder / 'subject_1.mat' #only looks at the first subject file, somewhere it said all the timings were the same so this should be safe
mat_contents = sio.loadmat(mat_file)
for count, row in enumerate(mat_contents['words'][0]):
    word_value = row[0][0][0][0]
    time = row[1][0][0]
    word_tuple = (word_value, time, feature_matrix[count,:])
    words_info.append(word_tuple)

## Align fmri and word features

In [5]:
#class for storing all the information for each sample

class sample: 
    def __init__(self, subj_id, time, input_voxels, output_voxels, input_words, input_word_features, output_words, output_word_features):
        self.subj_id = subj_id #id of subject
        
        self.time = time #time at which the scan occurred
        
        assert input_voxels.shape[3] == 4, "input_voxels does not contain 4 scans"
        self.input_voxels = input_voxels #2d array of 4 TRs of voxels at time of scan
        
        assert output_voxels.shape[3] == 4, "output_voxels does not contain 4 scans"
        self.output_voxels = output_voxels #2d array of 4 TRs of voxels associated with next 4 words
        
        assert len(input_words) == 4, "input_words does not contain 4 words"
        self.input_words = input_words #list of 4 words associated with scan
        
        assert len(output_words) == 4, "output_words does not contain 4 words"
        self.output_words = output_words #list of 4 words that come after scan
        
        assert input_word_features.shape[0] == 4, "input_word_features does not contain 4 words"
        self.input_word_features = input_word_features #np array of size (4,nFeatures) storing the features for the 4 words
        
        assert output_word_features.shape[0] == 4, "output_word_features does not contain 4 words"
        self.output_word_features = output_word_features #np array of size (4,nFeatures) storing the features for the 4 words afterwards
    
    def get_subj_id(self):
        return self.subj_id
    
    def get_time(self):
        return self.time
    
    def get_input_voxels(self):
        return self.input_voxels
    
    def get_output_voxels(self):
        return self.output_voxels
    
    def get_input_words(self):
        return self.input_words
    
    def get_input_word_features(self):
        return self.input_word_features
    
    def get_output_words(self):
        return self.output_words
    
    def get_output_word_features(self):
        return self.output_word_features

In [6]:
subjects_samples = [[] for i in range(NUM_SUBJS)] #stores lists of all the samples for each subject

#still working on this, need to deal with the issue where a rest happens 
word_count = 0
while word_count < len(words_info) - 8:
    #gets the 4 input words, and the 4 consecutive words while verifying they were read in sequence
    scan_words = []
    start_time = words_info[word_count][1]
    in_sequence = True #tracks if the words are in sequence or not
    for i in range(8):
        word_info = words_info[word_count + i]
        if word_info[1] != start_time + 0.5*i:
            #if some of the words are not in sequence, skip forward 1 word after innter loop
            in_sequence = False
        scan_words.append(word_info[0])
    if not in_sequence:
        word_count +=1
        continue
    #gets word features for input and output words
    input_word_features = feature_matrix[word_count:word_count+4, :]
    output_word_features = feature_matrix[word_count+4:word_count+8,:]
    #gets index of fmri that comes 2 seconds after reading first word
    fmri_time = start_time + 2 #effect of reading words is assumed to start 2 seconds after and end 8 seconds after
    fmri_index = fmri_time//2 #since a scan happens every two seconds, the index is the time divided by 2
    if not isinstance(fmri_index, np.int32):
        #if the first word is not aligned with the fmri scan (i.e. its not the first word in a TR)
        word_count += 1
        continue
    for count, subject in enumerate(subjects_fmri):
        new_sample = sample(count, 
                            start_time, 
                            subject[:,:,:,fmri_index:fmri_index+4], #gets the scans 2,4,6,8 seconds after reading
                            subject[:,:,:,fmri_index+2:fmri_index+6], #gets the scans 4,6,8,10 seconds after reading
                            scan_words[0:4], 
                            input_word_features,
                            scan_words[4:8],
                            output_word_features)
        subjects_samples[count].append(new_sample)
    print("Created sample:")
    print("\tScan time:", str(start_time))
    print("\tInput words:", str(scan_words[0:4]))
    print("\tOutput_words", str(scan_words[4:8]))
    #if successful, skip forward to the next set of 4 words
    word_count += 4

print("Total number of samples:", str(len(subjects_samples[0])))

Created sample:
	Scan time: 20
	Input words: ['Harry', 'had', 'never', 'believed']
	Output_words ['he', 'would', 'meet', 'a']
Created sample:
	Scan time: 22
	Input words: ['he', 'would', 'meet', 'a']
	Output_words ['boy', 'he', 'hated', 'more']
Created sample:
	Scan time: 24
	Input words: ['boy', 'he', 'hated', 'more']
	Output_words ['than', 'Dudley,', 'but', 'that']
Created sample:
	Scan time: 26
	Input words: ['than', 'Dudley,', 'but', 'that']
	Output_words ['was', 'before', 'he', 'met']
Created sample:
	Scan time: 28
	Input words: ['was', 'before', 'he', 'met']
	Output_words ['Draco', 'Malfoy.', 'Still,', 'first-year']
Created sample:
	Scan time: 30
	Input words: ['Draco', 'Malfoy.', 'Still,', 'first-year']
	Output_words ['Gryffindors', 'only', 'had', 'Potions']
Created sample:
	Scan time: 32
	Input words: ['Gryffindors', 'only', 'had', 'Potions']
	Output_words ['with', 'the', 'Slytherins,', 'so']
Created sample:
	Scan time: 34
	Input words: ['with', 'the', 'Slytherins,', 'so']
	Out

	Input words: ['Quidditch,', 'Potter?"', 'he', 'asked']
	Output_words ['excitedly.', '+', '"Wood\'s', 'captain']
Created sample:
	Scan time: 1182
	Input words: ['excitedly.', '+', '"Wood\'s', 'captain']
	Output_words ['of', 'the', 'Gryffindor', 'team,"']
Created sample:
	Scan time: 1184
	Input words: ['of', 'the', 'Gryffindor', 'team,"']
	Output_words ['Professor', 'McGonagall', 'explained.', '+']
Created sample:
	Scan time: 1186
	Input words: ['Professor', 'McGonagall', 'explained.', '+']
	Output_words ['"He\'s', 'just', 'the', 'build']
Created sample:
	Scan time: 1188
	Input words: ['"He\'s', 'just', 'the', 'build']
	Output_words ['for', 'a', 'Seeker,', 'too,"']
Created sample:
	Scan time: 1190
	Input words: ['for', 'a', 'Seeker,', 'too,"']
	Output_words ['said', 'Wood,', 'now', 'walking']
Created sample:
	Scan time: 1192
	Input words: ['said', 'Wood,', 'now', 'walking']
	Output_words ['around', 'Harry', 'and', 'staring']
Created sample:
	Scan time: 1194
	Input words: ['around', 'Har

	Output_words ['+', '"NOTHING!', 'Ha', 'haaa!']
Created sample:
	Scan time: 2388
	Input words: ['+', '"NOTHING!', 'Ha', 'haaa!']
	Output_words ['Told', 'you', 'I', "wouldn't"]
Created sample:
	Scan time: 2390
	Input words: ['Told', 'you', 'I', "wouldn't"]
	Output_words ['say', 'nothing', 'if', 'you']
Created sample:
	Scan time: 2392
	Input words: ['say', 'nothing', 'if', 'you']
	Output_words ["didn't", 'say', 'please!', 'Ha']
Created sample:
	Scan time: 2394
	Input words: ["didn't", 'say', 'please!', 'Ha']
	Output_words ['ha!', 'Haaaaaa!"', 'And', 'they']
Created sample:
	Scan time: 2396
	Input words: ['ha!', 'Haaaaaa!"', 'And', 'they']
	Output_words ['heard', 'the', 'sound', 'of']
Created sample:
	Scan time: 2398
	Input words: ['heard', 'the', 'sound', 'of']
	Output_words ['Peeves', 'whooshing', 'away', 'and']
Created sample:
	Scan time: 2400
	Input words: ['Peeves', 'whooshing', 'away', 'and']
	Output_words ['Filch', 'cursing', 'in', 'rage.']
Created sample:
	Scan time: 2402
	Input w

## Initialize model

In [135]:
class MLModel(nn.Module):
    def __init__(self, fmri_shape, num_word_features, num_outputs):
        super(MLModel, self).__init__()
        
        self.num_word_features = num_word_features
        
        self.conv1 = nn.Conv3d(1, 8, 7, stride=2)
        self.conv2 = nn.Conv3d(1, 16, 5, stride=2)
        self.conv3 = nn.Conv3d(1, 32, 3, stride=2)
        #self.fc = nn.Linear(2560, num_outputs)
        self.fc = nn.Linear(80, num_outputs)
        
        self.max_pool = nn.MaxPool3d(3)
        
        self.dropout = nn.Dropout()
        
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.soft = nn.Softmax(dim=0)
        
        self.word_fc = nn.Linear(num_word_features, num_outputs)
        

    def forward(self, x_fmri, x_words):
        if self.num_word_features == 0:
            fmri_out = torch.unsqueeze(x_fmri, 0)

            fmri_out = self.conv1(fmri_out)
            fmri_out = torch.mean(fmri_out, dim=0, keepdim=True)

            fmri_out = self.conv2(fmri_out)
            fmri_out = torch.mean(fmri_out, dim=0, keepdim=True)
            fmri_out = self.dropout(fmri_out)

            fmri_out = self.conv3(fmri_out)
            fmri_out = torch.mean(fmri_out, dim=0, keepdim=True)
            fmri_out = self.dropout(fmri_out)
            #print(fmri_out.flatten().shape)

            fmri_out = self.fc(fmri_out.flatten())
            return fmri_out
        else:
            out = self.word_fc(x_words)
            return out

In [203]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv3d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv3d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [206]:
#https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv3d(1, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.ReLU())
        self.maxpool = nn.MaxPool3d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool3d(2, stride=1) #i changed 7 to 2
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes, kernel_size=1, stride=stride),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x)
        #print(x.shape)
        x = self.maxpool(x)
        #print(x.shape)
        x = self.layer0(x)
        #print(x.shape)
        x = self.layer1(x)
        #print(x.shape)
        x = self.layer2(x)
        #print(x.shape)
        x = self.layer3(x)
        #print(x.shape)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = x.flatten() #i added this
        x = self.fc(x)

        return x

## Build samples and separate into folds

In [192]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [193]:
subject_0_samples = subjects_samples[0] #only using the first subjects samples currently

fmri_shape = subject_0_samples[0].get_input_voxels().shape
fmri_samples = np.zeros([len(subject_0_samples)] + list(fmri_shape[0:3]))

word_feature_size = subject_0_samples[0].get_input_word_features().shape[1]
word_samples = np.zeros((len(subject_0_samples), word_feature_size*5))
for i in range(len(subject_0_samples)):
    fmri_samples[i] = subject_0_samples[i].get_input_voxels()[:,:,:,2] #get feature 6 seconds after reading 4 words
    word_samples[i, :word_feature_size*4] = subject_0_samples[i].get_input_word_features().flatten()
    word_samples[i, word_feature_size*4:] = np.sum(subject_0_samples[i].get_output_word_features(), 0)

In [194]:
NUM_FOLDS = 3
fmri_samples, word_samples = unison_shuffled_copies(fmri_samples, word_samples) #shuffles rows
print(fmri_samples.shape)
print(word_samples.shape)
fmri_folds = np.array(np.split(fmri_samples, NUM_FOLDS))
word_folds = np.array(np.split(word_samples, NUM_FOLDS))
print(fmri_folds.shape)
print(word_folds.shape)
# for i in range(word_feature_size*4, word_feature_size*5):
#     print("i:", i)
#     print(np.nonzero(word_samples[:, i])[0].shape[0])
print(np.nonzero(word_samples[:, 919]))
print(word_samples[np.nonzero(word_samples[:, 919]), 919])

(1287, 53, 60, 50)
(1287, 975)
(3, 429, 53, 60, 50)
(3, 429, 975)
(array([   3,    4,   14,   16,   17,   24,   25,   28,   29,   31,   32,
         33,   42,   53,   55,   77,   80,   95,   96,  108,  139,  158,
        159,  164,  175,  177,  179,  188,  189,  198,  200,  208,  212,
        218,  222,  223,  225,  227,  230,  232,  237,  245,  259,  263,
        267,  272,  288,  290,  291,  293,  296,  297,  318,  326,  333,
        343,  346,  350,  355,  359,  366,  370,  372,  374,  377,  386,
        395,  397,  403,  416,  421,  422,  425,  432,  435,  449,  452,
        468,  470,  474,  481,  498,  502,  508,  510,  511,  516,  523,
        530,  537,  540,  547,  549,  551,  560,  561,  576,  580,  592,
        609,  611,  613,  635,  637,  646,  653,  655,  657,  660,  666,
        673,  677,  687,  688,  697,  703,  705,  711,  712,  713,  714,
        731,  733,  737,  741,  749,  759,  763,  764,  768,  777,  780,
        785,  796,  804,  810,  812,  828,  834,  839,  8

## Train model

In [217]:
#performs stochastic gradient descent for num_epoch epochs
def train_model(model, fmri_samples, word_samples, num_word_features, output_idxs, alpha=1e-7, weight_decay = 0.001, momentum=0.9, num_epochs=50):
    print("\tTraining for ", num_epochs, " epochs:")
    optimizer = torch.optim.SGD(model.parameters(), lr=alpha, weight_decay=weight_decay, momentum=momentum)
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.data)
    loss_fn = nn.MSELoss()
    for epoch in range(num_epochs):
        epoch_loss = 0
        #randomly shuffle samples 
        fmri_samples, word_samples = unison_shuffled_copies(fmri_samples, word_samples)
        for sample_idx in range(fmri_samples.shape[0]):
            optimizer.zero_grad()
            fmri_x = torch.tensor(fmri_samples[sample_idx]).float().to(device)
            word_x = torch.tensor(word_samples[sample_idx, :num_word_features]).float().to(device)
            y = torch.tensor(word_samples[sample_idx, output_idxs]).float().to(device)
            #pred = model(fmri_x, word_x)
            pred = model(torch.unsqueeze(fmri_x, 0))
            loss = loss_fn(pred, y)
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()
        if epoch % 1 == 0:
            print("\t\tEpoch: ", epoch, ", Train Loss: ", epoch_loss)
        if epoch % 1 == 0:
            train_accuracy = get_accuracy(model, fmri_samples, word_samples, num_word_features, output_idxs)
            print("\t\tEpoch: ", epoch, ", Train Accuracy: ", train_accuracy)
    print("\t\tEpoch: ", epoch, ", Train Loss: ", epoch_loss)

In [196]:
#gets the model accuracy by getting the prediction for a sample and seeing 
#if it closer to the labels of that sample or another randomly selected sample
def get_accuracy(model, fmri_samples, word_samples, num_word_features, output_idxs):
    correct = 0
    for sample_idx in range(fmri_samples.shape[0]):
        #gets the input and labels from current sample
        correct_fmri_x = torch.tensor(fmri_samples[sample_idx]).float().to(device)
        correct_word_x = torch.tensor(word_samples[sample_idx, :num_word_features]).float().to(device)
        correct_labels = torch.tensor(word_samples[sample_idx, output_idxs]).float().to(device)
        #gets labels from a random sample
        rand = random.randint(0, word_samples.shape[0]-1)
        random_labels = torch.tensor(word_samples[rand,output_idxs]).float().to(device)
        #gets prediction on current sample and computes euclidean distances from both correct and random labels
        #pred = model(correct_fmri_x, correct_word_x)
        pred = model(torch.unsqueeze(correct_fmri_x, 0))
        correct_distance = torch.linalg.norm(correct_labels - pred)
        random_distance = torch.linalg.norm(random_labels - pred)
        #if distance to correct labels is less than distance to random labels then the prediction is considered correct
        if correct_distance <= random_distance:
            correct += 1
    return correct / word_samples.shape[0]

In [199]:
#for each fold split, creates a model and trains it on n-1 folds and then tests it on the last fold
def cross_validate(fmri_folds, word_folds, num_folds, num_fmri_features, num_word_features, output_idxs, num_epochs=50):
    for i in range(num_folds):
        print("Fold:", i)
        loss_fn = nn.MSELoss()
        #gets all folds except one and reshapes them into a 2d array
        fmri_train_samples = np.delete(fmri_folds, i, axis=0)
        fmri_shape = fmri_train_samples.shape
        fmri_train_samples = np.reshape(fmri_train_samples, (fmri_shape[1]*(num_folds-1), fmri_shape[2], fmri_shape[3], fmri_shape[4]))
        word_train_samples = np.delete(word_folds, i, axis=0)
        word_shape = word_train_samples.shape
        word_train_samples = np.reshape(word_train_samples, (word_shape[1]*(num_folds-1), word_shape[2]))
        fmri_test_samples = fmri_folds[i]
        word_test_samples = word_folds[i]
        
        model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=len(output_idxs)).to(device)
        #model = MLModel(fmri_folds.shape[2:], num_word_features, len(output_idxs)).to(device)
        train_model(model, fmri_train_samples, word_train_samples, num_word_features, output_idxs, num_epochs=num_epochs)
            
        train_loss = 0
        for sample_idx in range(fmri_train_samples.shape[0]):
            fmri_x = torch.tensor(fmri_train_samples[sample_idx]).float().to(device)
            word_x = torch.tensor(word_train_samples[sample_idx, :num_word_features]).float().to(device)
            y = torch.tensor(word_train_samples[sample_idx, output_idxs]).float().to(device)
            #pred = model(fmri_x, word_x)
            pred = model(torch.unsqueeze(fmri_x, 0))
            loss = loss_fn(pred, y)
            train_loss += loss.item()
        train_accuracy = get_accuracy(model, fmri_train_samples, word_train_samples, num_word_features, output_idxs)
        print("\tTrain Loss: ", train_loss, ", Train Accuracy: ", train_accuracy)
        
        test_loss = 0
        for sample_idx in range(fmri_test_samples.shape[0]):
            fmri_x = torch.tensor(fmri_test_samples[sample_idx]).float().to(device)
            word_x = torch.tensor(word_test_samples[sample_idx, :num_word_features]).float().to(device)
            y = torch.tensor(word_test_samples[sample_idx, output_idxs]).float().to(device)
            #pred = model(fmri_x, word_x)
            pred = model(torch.unsqueeze(fmri_x, 0))
            loss = loss_fn(pred, y)
            test_loss += loss.item()
        test_accuracy = get_accuracy(model, fmri_test_samples, word_test_samples, num_word_features, output_idxs)
        print("\tTest Loss: ", test_loss, ", Test Accuracy: ", test_accuracy)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#output_idxs = [*range(word_feature_size*4 + 102, word_feature_size*4 + 146)]
output_idxs = [*range(word_feature_size*4, word_feature_size*5)]
#output_idxs = [919] #try to predict the "harry" feature (i assume its the most commonly read one)
print(output_idxs)
cross_validate(fmri_folds, word_folds, NUM_FOLDS, fmri_shape, 0, output_idxs, num_epochs=1000)

cuda
[780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905, 906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974]
Fold: 0
	Training f

		Epoch:  75 , Train Loss:  7002.6707347258925
		Epoch:  75 , Train Accuracy:  0.48951048951048953
		Epoch:  76 , Train Loss:  7001.251033373177
		Epoch:  76 , Train Accuracy:  0.5011655011655012
		Epoch:  77 , Train Loss:  7013.062570847571
		Epoch:  77 , Train Accuracy:  0.5116550116550117
		Epoch:  78 , Train Loss:  7007.623507089913
		Epoch:  78 , Train Accuracy:  0.49533799533799533
